In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
print(module_path)
from bigbio.dataloader import BigBioConfigHelpers
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, TokenClassificationPipeline
from transformers_interpret.evaluation.input_pre_processor import InputPreProcessor, get_labels_from_dataset
import math

C:\Users\richa\Documents\GitHub\transformers-interpret


In [2]:
def map_to_string(x):
    return id2label[x]

In [13]:
conhelps = BigBioConfigHelpers()
# dataset_name = 'bc5cdr_bigbio_kb'  # 2 classes, short to medium sentence length, Disease
dataset_name = 'euadr_bigbio_kb'  # 5 classes, short to medium sentence length, Diseases & Disorders
# dataset_name = 'cadec_bigbio_kb'  # 5 classes, shortest documents, forum posts, Disease
# dataset_name = 'scai_disease_bigbio_kb'  # 2 classes, long documents, DISEASE
# dataset_name = 'ncbi_disease_bigbio_kb'
# dataset_name = 'verspoor_2013_bigbio_kb'
dataset = conhelps.for_config_name(dataset_name).load_dataset()
print(dataset['train'][0])

Reusing dataset euadr (C:\Users\richa\.cache\huggingface\datasets\euadr\euadr_bigbio_kb\1.0.0\cf05fbfbed982fab21e5d4eea1e3dd6888f5b2065c8a803ff8519c6df02b2b1a)


  0%|          | 0/1 [00:00<?, ?it/s]

{'id': '0', 'document_id': '0', 'passages': [{'id': '1', 'type': 'title', 'text': ['A sequential procedure for monitoring clinical trials against historical controls.'], 'offsets': [[0, 82]]}, {'id': '2', 'type': 'abstract', 'text': ['In this paper, we develop a sequential procedure to monitor clinical trials against historical controls. When there is a strong ethical concern about randomizing patients to existing treatment because biological and medical evidence suggests that the new treatment is potentially superior to the existing one, or when the enrollment is too limited for randomization of subjects into experimental and control groups, one can monitor the trial sequentially against historical controls if the historical data with required quality and sample size are available to form a valid reference for the trial. This design of trial is sometimes the only alternative to a randomized phase III trial design that is intended but not feasible in situations such as above. Monitorin

In [4]:
# huggingface_model = 'dbmdz/electra-large-discriminator-finetuned-conll03-english'
# huggingface_model = 'dslim/bert-base-NER'
huggingface_model = 'Jean-Baptiste/roberta-large-ner-english'

tokenizer: AutoTokenizer = AutoTokenizer.from_pretrained(huggingface_model)
additional_tokenizers = []

In [5]:
label2id, id2label = get_labels_from_dataset(dataset)
print(label2id)
pre_processor = InputPreProcessor(tokenizer, additional_tokenizers, label2id)

{'O': 0, 'B-Chemicals & Drugs': 1, 'I-Chemicals & Drugs': 2, 'B-Diseases & Disorders': 3, 'I-Diseases & Disorders': 4, 'B-Genes & Molecular Sequences': 5, 'I-Genes & Molecular Sequences': 6, 'B-SNP & Sequence variations': 7, 'I-SNP & Sequence variations': 8}


In [6]:
tokenized_datasets = dataset.map(lambda a: pre_processor(a))

Parameter 'function'=<function <lambda> at 0x0000029A84894B80> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/300 [00:00<?, ?ex/s]

In [7]:
from transformers import TokenClassificationPipeline
finetuned_huggingface_model = './trained_models/roberta_euadr'
# if huggingface_model == 'Jean-Baptiste/roberta-large-ner-english' and dataset_name == 'bc5cdr_bigbio_kb':
#     finetuned_huggingface_model = './trained_models/Roberta_BC5'
# else:
#     finetuned_huggingface_model = './trained_models/Electra_SCAI'
model: AutoModelForTokenClassification = AutoModelForTokenClassification.from_pretrained(finetuned_huggingface_model, local_files_only=True)
model.config.id2label, model.config.label2id = id2label, label2id
print(model.config.id2label, model.config.label2id)
pipeline = TokenClassificationPipeline(model=model, tokenizer=tokenizer)

{0: 'O', 1: 'B-Chemicals & Drugs', 2: 'I-Chemicals & Drugs', 3: 'B-Diseases & Disorders', 4: 'I-Diseases & Disorders', 5: 'B-Genes & Molecular Sequences', 6: 'I-Genes & Molecular Sequences', 7: 'B-SNP & Sequence variations', 8: 'I-SNP & Sequence variations'} {'O': 0, 'B-Chemicals & Drugs': 1, 'I-Chemicals & Drugs': 2, 'B-Diseases & Disorders': 3, 'I-Diseases & Disorders': 4, 'B-Genes & Molecular Sequences': 5, 'I-Genes & Molecular Sequences': 6, 'B-SNP & Sequence variations': 7, 'I-SNP & Sequence variations': 8}


In [8]:
if 'test' in dataset:
    test_dataset = tokenized_datasets["test"]
else:
    dataset_length = len(dataset["train"])
    shuffled_dataset = tokenized_datasets["train"].shuffle(seed=42)
    test_dataset = shuffled_dataset.select(range(math.floor(dataset_length * 0.8), math.floor(dataset_length * 0.9)))
print(len(test_dataset))

30


In [12]:
for doc_i, doc in enumerate(test_dataset):
    if doc_i < 0:
        continue
    
    if doc_i >= 20:
        break
    preds = pipeline(doc['text'])
#     print(preds)
#     break
    pred_labels = [0 for _ in range(len(doc['labels']))]
    for i in preds:
        pred_labels[i['index']] = label2id[i['entity']]
    gold_labels = doc['labels']
#     print(pred_labels, gold_labels)
    print(doc['document_id'])
    for tok_i, labels in enumerate(zip(pred_labels, gold_labels, tokenizer.batch_decode(doc['input_ids']))):
        pred_label = labels[0]
        gold_label = labels[1]
        token_string = labels[2]
        if token_string in ['<pad>', '[PAD]']:
            break
        if gold_label != pred_label:
            print(f"{tok_i} {token_string}, {id2label[pred_label]} {'(Gold label: {})'.format(id2label[gold_label]) if pred_label != gold_label else ''}")
#         if pred_label != gold_label:
#             print(token_string, id2label[pred_label], 'instead of', id2label[gold_label])
#     print('len(doc[\'entities\'])', len(doc['entities']))
#     for e in doc['entities']:
#         if not e['type'] == '':
#             print(e)
    print('\n\n\n')

6677
169  rash, B-Diseases & Disorders (Gold label: O)




9451




12213




16894
19  inhibitors, B-Chemicals & Drugs (Gold label: O)
20  of, I-Chemicals & Drugs (Gold label: O)
40  receptors, I-Genes & Molecular Sequences (Gold label: O)
44  tumor, O (Gold label: B-Diseases & Disorders)
57  EG, B-SNP & Sequence variations (Gold label: B-Genes & Molecular Sequences)
58 FR, I-SNP & Sequence variations (Gold label: I-Genes & Molecular Sequences)
59  gene, I-SNP & Sequence variations (Gold label: O)
60  polymorph, I-SNP & Sequence variations (Gold label: O)
61 ism, I-SNP & Sequence variations (Gold label: O)
70  pancreat, O (Gold label: B-Diseases & Disorders)
71 ic, O (Gold label: I-Diseases & Disorders)
72  ad, O (Gold label: I-Diseases & Disorders)
73 en, O (Gold label: I-Diseases & Disorders)
74 oc, O (Gold label: I-Diseases & Disorders)
75 arc, O (Gold label: I-Diseases & Disorders)
76 in, O (Gold label: I-Diseases & Disorders)
93  AA, O (Gold label: B-SNP & Sequence variations)
94

541
448  sub, B-Chemicals & Drugs (Gold label: O)
449 type, I-Chemicals & Drugs (Gold label: O)
450 -, I-Chemicals & Drugs (Gold label: O)
451 select, I-Chemicals & Drugs (Gold label: O)
452 ive, I-Chemicals & Drugs (Gold label: O)
453  PP, I-Chemicals & Drugs (Gold label: B-Chemicals & Drugs)




6281
80  SAR, B-Chemicals & Drugs (Gold label: B-Genes & Molecular Sequences)




2820




8313




3655




12146




9846




13209
10  contrast, B-Chemicals & Drugs (Gold label: I-Chemicals & Drugs)
125 aren, I-Chemicals & Drugs (Gold label: O)
126 ter, I-Chemicals & Drugs (Gold label: O)
127 al, I-Chemicals & Drugs (Gold label: O)




4




2413
7  aging, B-Diseases & Disorders (Gold label: O)
23  neuro, O (Gold label: B-Diseases & Disorders)
26 ative, O (Gold label: I-Diseases & Disorders)
27  diseases, B-Diseases & Disorders (Gold label: I-Diseases & Disorders)
49  mitochondrial, O (Gold label: B-SNP & Sequence variations)
50  DNA, O (Gold label: I-SNP & Sequence variations)
98  deficie